In [1]:
%reload_ext autoreload
%autoreload 2

from sarpu.data_processing import *
from sarpu.paths_and_names import *

import numpy as np
import pandas as pd
import requests

import sklearn.model_selection

# Input Data

In [2]:
# Names and locations
data_folder= "../../Data/"
data_name = "imagesegmentation"
url1="http://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.data"
url2="http://archive.ics.uci.edu/ml/machine-learning-databases/image/segmentation.test"

In [3]:
# Creation information
nb_splits = 5
test_size = 0.2

# Prepare folders

In [4]:
# Prepare folders
data_folder_original = original_data_path(data_folder,data_name)
!mkdir -p $data_folder_original
data_folder_processed = processed_data_path(data_folder,data_name)
!mkdir -p $data_folder_processed
data_folder_partitions = partitions_data_path(data_folder,data_name)
!mkdir -p $data_folder_partitions

# Download

In [5]:
#download and unzip data

unprocessed_data_path1 = os.path.join(data_folder_original,url1.split("/")[-1])
if not os.path.exists(unprocessed_data_path1):
    r = requests.get(url1, allow_redirects=True)
    open(unprocessed_data_path1, 'wb').write(r.content)
    
unprocessed_data_path2 = os.path.join(data_folder_original,url2.split("/")[-1])
if not os.path.exists(unprocessed_data_path2):
    r = requests.get(url2, allow_redirects=True)
    open(unprocessed_data_path2, 'wb').write(r.content)

In [6]:
#read data to pandas dataframe



#     1.  region-centroid-col:  the column of the center pixel of the region.
#     2.  region-centroid-row:  the row of the center pixel of the region.
#     3.  region-pixel-count:  the number of pixels in a region = 9.
#     4.  short-line-density-5:  the results of a line extractoin algorithm that 
#          counts how many lines of length 5 (any orientation) with
#          low contrast, less than or equal to 5, go through the region.
#     5.  short-line-density-2:  same as short-line-density-5 but counts lines
#          of high contrast, greater than 5.
#     6.  vedge-mean:  measure the contrast of horizontally
#          adjacent pixels in the region.  There are 6, the mean and 
#          standard deviation are given.  This attribute is used as
#         a vertical edge detector.
#     7.  vegde-sd:  (see 6)
#     8.  hedge-mean:  measures the contrast of vertically adjacent
#           pixels. Used for horizontal line detection. 
#     9.  hedge-sd: (see 8).
#     10. intensity-mean:  the average over the region of (R + G + B)/3
#     11. rawred-mean: the average over the region of the R value.
#     12. rawblue-mean: the average over the region of the B value.
#     13. rawgreen-mean: the average over the region of the G value.
#     14. exred-mean: measure the excess red:  (2R - (G + B))
#     15. exblue-mean: measure the excess blue:  (2B - (G + R))
#     16. exgreen-mean: measure the excess green:  (2G - (R + B))
#     17. value-mean:  3-d nonlinear transformation
#          of RGB. (Algorithm can be found in Foley and VanDam, Fundamentals
#          of Interactive Computer Graphics)
#     18. saturatoin-mean:  (see 17)
#     19. hue-mean:  (see 17)



header = [
    "class",
    "region-centroid-col",
    "region-centroid-row",
    "region-pixel-count",
    "short-line-density-5",
    "short-line-density-2",
    "vedge-mean",
    "vegde-sd",
    "hedge-mean",
    "hedge-sd",
    "intensity-mean",
    "rawred-mean",
    "rawblue-mean",
    "rawgreen-mean",
    "exred-mean",
    "exblue-mean",
    "exgreen-mean",
    "value-mean",
    "saturatoin-mean",
    "hue-mean",
]

multival = []


# df = pd.read_csv(data_path, names=header).replace('?', np.NaN).dropna()

df1 = pd.read_csv(unprocessed_data_path1, skiprows=5, names=header)
df2 = pd.read_csv(unprocessed_data_path2, skiprows=5, names=header)
df = pd.concat([df1,df2], ignore_index=True)
df = df.drop("region-pixel-count",axis=1)


df

,class,region-centroid-col,region-centroid-row,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturatoin-mean,hue-mean
0,BRICKFACE,140.0,125.0,0.000000,0.000000,0.277778,0.062963,0.666667,0.311111,6.185185,7.333334,7.666666,3.555556,3.444444,4.444445,-7.888889,7.777778,0.545635,-1.121818
1,BRICKFACE,188.0,133.0,0.000000,0.000000,0.333333,0.266667,0.500000,0.077778,6.666666,8.333334,7.777778,3.888889,5.000000,3.333333,-8.333333,8.444445,0.538580,-0.924817
2,BRICKFACE,105.0,139.0,0.000000,0.000000,0.277778,0.107407,0.833333,0.522222,6.111111,7.555555,7.222222,3.555556,4.333334,3.333333,-7.666666,7.555555,0.532628,-0.965946
3,BRICKFACE,34.0,137.0,0.000000,0.000000,0.500000,0.166667,1.111111,0.474074,5.851852,7.777778,6.444445,3.333333,5.777778,1.777778,-7.555555,7.777778,0.573633,-0.744272
4,BRICKFACE,39.0,111.0,0.000000,0.000000,0.722222,0.374074,0.888889,0.429629,6.037037,7.000000,7.666666,3.444444,2.888889,4.888889,-7.777778,7.888889,0.562919,-1.175773
5,BRICKFACE,16.0,128.0,0.000000,0.000000,0.500000,0.077778,0.666667,0.311111,5.555555,6.888889,6.666666,3.111111,4.000000,3.333333,-7.333334,7.111111,0.561508,-0.985811
6,BRICKFACE,26.0,67.0,0.111111,0.000000,1.000000,0.888890,2.444445,3.185185,20.000000,19.555555,25.888890,14.555555,-1.333333,17.666666,-16.333334,25.888890,0.436939,-1.623202
7,BRICKFACE,14.0,110.0,0.000000,0.000000,1.722222,5.351850,2.666667,1.022223,17.925926,18.888890,21.444445,13.444445,2.888889,10.555555,-13.444445,21.444445,0.368848,-1.345096
8,BRICKFACE,11.0,108.0,0.000000,0.000000,1.333333,0.800000,1.388889,0.951852,17.666666,19.000000,21.111110,12.888889,4.000000,10.333333,-14.333333,21.111110,0.388756,-1.302133
9,BRICKFACE,85.0,101.0,0.000000,0.000000,1.333333,1.288888,1.277778,1.218518,21.296297,21.222221,26.777779,15.888889,-0.222222,16.444445,-16.222221,26.777779,0.404792,-1.558599


In [7]:
#class distribution

df["class"].value_counts()


CEMENT       330
PATH         330
SKY          330
WINDOW       330
BRICKFACE    330
FOLIAGE      330
GRASS        330
Name: class, dtype: int64

In [8]:
# Make nature positive class

df.loc[df["class"]=="BRICKFACE","class"]=0
df.loc[df["class"]=="CEMENT","class"]=0
df.loc[df["class"]=="WINDOW","class"]=0
df.loc[df["class"]=="PATH","class"]=0

df.loc[df["class"]=="SKY","class"]=1
df.loc[df["class"]=="GRASS","class"]=1
df.loc[df["class"]=="FOLIAGE","class"]=1

# df

In [9]:
#binarize multivalued features

for column in multival:
    values = list(set(df[column]))
    if len(values)>2:
        df = binarize(df, column)
    elif len(values)==2:
        df.loc[df[column]==values[0],column]=-1
        df.loc[df[column]==values[1],column]=1
    else: # drop useless features
        print(column, values)
        df=df.drop(column, axis=1)

# df

In [10]:
#normalize
for column in df.columns.values:
    df[column]=pd.to_numeric(df[column])
    
    

normalized_df=(df.astype(float)-df.min())/(df.max()-df.min())*2-1
normalized_df["class"] = df["class"]
df = normalized_df
# df

In [11]:
#move class to back

cols = list(df.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('class')) #Remove class from list
df = df[cols+['class']]

df

,region-centroid-col,region-centroid-row,short-line-density-5,short-line-density-2,vedge-mean,vegde-sd,hedge-mean,hedge-sd,intensity-mean,rawred-mean,rawblue-mean,rawgreen-mean,exred-mean,exblue-mean,exgreen-mean,value-mean,saturatoin-mean,hue-mean,class
0,0.098814,-0.050000,-1.000000,-1.0,-0.980989,-0.999873,-0.970186,-0.999551,-0.913762,-0.893031,-0.898380,-0.950117,0.783582,-0.642353,-0.111954,-0.896907,0.091270,-0.354552,0
1,0.478261,0.016667,-1.000000,-1.0,-0.977186,-0.999462,-0.977640,-0.999888,-0.907049,-0.878444,-0.896907,-0.945440,0.835821,-0.665882,-0.127135,-0.888071,0.077160,-0.288407,0
2,-0.177866,0.066667,-1.000000,-1.0,-0.980989,-0.999783,-0.962733,-0.999247,-0.914795,-0.889789,-0.904271,-0.950117,0.813433,-0.665882,-0.104364,-0.899853,0.065256,-0.302216,0
3,-0.739130,0.050000,-1.000000,-1.0,-0.965779,-0.999664,-0.950311,-0.999316,-0.918409,-0.886548,-0.914580,-0.953235,0.861940,-0.698824,-0.100569,-0.896907,0.147266,-0.227787,0
4,-0.699605,-0.166667,-1.000000,-1.0,-0.950570,-0.999246,-0.960248,-0.999380,-0.915828,-0.897893,-0.898380,-0.951676,0.764925,-0.632941,-0.108159,-0.895434,0.125838,-0.372667,0
5,-0.881423,-0.025000,-1.000000,-1.0,-0.965779,-0.999843,-0.970186,-0.999551,-0.922541,-0.899514,-0.911635,-0.956352,0.802239,-0.665882,-0.092979,-0.905744,0.123016,-0.308886,0
6,-0.802372,-0.533333,-0.333333,-1.0,-0.931559,-0.998207,-0.890683,-0.995405,-0.721146,-0.714749,-0.656848,-0.795791,0.623134,-0.362353,-0.400379,-0.656848,-0.126122,-0.522896,0
7,-0.897233,-0.175000,-1.000000,-1.0,-0.882129,-0.989207,-0.880745,-0.998525,-0.750065,-0.724473,-0.715758,-0.811380,0.764925,-0.512941,-0.301708,-0.715758,-0.262303,-0.429519,0
8,-0.920949,-0.191667,-1.000000,-1.0,-0.908745,-0.998387,-0.937888,-0.998627,-0.753679,-0.722853,-0.720177,-0.819174,0.802239,-0.517647,-0.332068,-0.720177,-0.222487,-0.415094,0
9,-0.335968,-0.250000,-1.000000,-1.0,-0.908745,-0.997401,-0.942857,-0.998242,-0.703073,-0.690438,-0.645066,-0.777085,0.660448,-0.388235,-0.396584,-0.645066,-0.190416,-0.501205,0


In [12]:
# make numpy array

import numpy as np

xy = df.values

x = xy[:,:-1].astype(float)
y = xy[:,-1].astype(int)

x_pos = x[y==1]
x_neg = x[y==0]


In [13]:
#Save data and true classes
np.savetxt(data_path(data_folder, data_name), x)
np.savetxt(classlabels_path(data_folder, data_name), y,fmt='%d')

# Different dataset partitions (train/test and class prior)

In [14]:
sss = sklearn.model_selection.StratifiedShuffleSplit(n_splits=nb_splits, test_size=test_size, random_state=0)
splits = list(sss.split(x,y))

In [15]:
#save partitions. 0 means not in data, 1 means in train partition, 2 means in test partition

for i, (train,test) in enumerate(splits):
    partition = np.zeros_like(y,dtype=int)
    partition[train]=1
    
    partition[test]=2    
    np.savetxt(partition_path(data_folder,data_name, i), partition, fmt='%d')
   